In [ ]:
import uproot
import matplotlib.pyplot as plt 
import pandas as pd 
import numpy as np 
import matplotlib as mpl

from util import *
import var
import cut
import data
import hist

import importlib

from pyanalib import panda_helpers

from scipy import optimize

In [ ]:
plt.rcParams.update({'font.size': 16})
# mpl.rcParams['lines.linewidth'] = 4.

dosave = True
savedir = "plots_2_25_24/"

prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

In [ ]:
importlib.reload(var)
importlib.reload(cut)
importlib.reload(data)
importlib.reload(hist)
importlib.reload(panda_helpers)

In [ ]:
filedir = "/icarus/data/users/gputnam/DMCP2023G/mc-F-spectra/"
filetype = "F-CohLike_%s_evt_spectrum.df"
# filetype = "F2-Higgs_M260_%s_evt_spectrum.df"


masses = [220, 240, 260, 280, 300, 320, 340]

filetypes = ["F2-Higgs_M%i_%s_evt_spectrum.df" % (M, "%s") for M in masses] +\
    ["F-CohLike_%s_evt_spectrum.df", "F-CohLike_%s_loose_evt_spectrum.df"]

ftype_names = ["$M_S: $ %i MeV" % M for M in masses] + ["Coh-like", "Coh-like Loose"]

In [ ]:
nominal = "nom"
nominal_coh = "nom-all"

variations = [
    "ind0nom",
    "noiselhi",
    "ind1bin0",
    "ind1bin14",
    "sce2x",
    "ind0glo",
    "ind0ghi",
    
    "nom_gainlo",
    "nom_gainhi",
    "nom_callo",
    "nom_calhi",
    "nom_mcslo",
    "nom_mcshi"
]

vnames = [
    "Front Ind. W.C.",
    "Noise 1.1x",
    "Middle Ind. Opaque",
    "Middle Ind. Transparent",
    "SCE 2x",
    "Ind. Gain 0.85x",
    "Ind. Gain 1.15x",
    
    "Cal. Gain 1.01x",
    "Cal. Gain 0.99x",
    "Cal. dE/dx Low",
    "Cal. dE/dx High",
    "MCS Low",
    "MCS High"
]

In [ ]:
cvdfs = [pd.read_hdf(filedir + ft % (nominal if i < len(masses) else nominal_coh)) for i,ft in enumerate(filetypes)]
cv_escale_dfs = [pd.read_hdf(filedir + ft % nominal) for i,ft in enumerate(filetypes)]

vardfs = [[pd.read_hdf(filedir + ft % v) for v in variations] for ft in filetypes]

In [ ]:
# Divide out the cv-weights
for i in range(len(cvdfs)):
    cvdfs[i].weight /= cvdfs[i].cvweight

for i in range(len(cv_escale_dfs)):
    cv_escale_dfs[i].weight /= cv_escale_dfs[i].cvweight
    
for i in range(len(vardfs)):
    for j in range(len(vardfs[i])):
        vardfs[i][j].weight /= vardfs[i][j].cvweight

In [ ]:
allratios = []
allratio_errs = []

means = []
mean_errs = []

for iv, v in enumerate(variations):
    plt.figure(iv)
            
    ratios = []
    ratio_errs = []
    
    this_cvdfs = cvdfs if iv < 7 else cv_escale_dfs
    
    for ift, ft in enumerate(filetypes):
        cv_evt = this_cvdfs[ift].weight.sum()
        cv_evt_var = (this_cvdfs[ift].weight**2).sum()
        
        var_evt = vardfs[ift][iv].weight.sum()
        var_evt_var = (vardfs[ift][iv].weight**2).sum()

        ratio =  var_evt/cv_evt

        ratio_err = np.sqrt(var_evt_var/cv_evt**2 + cv_evt_var*var_evt**2/cv_evt**4)

        ratios.append(ratio)
        ratio_errs.append(ratio_err)
        
    ratios = np.array(ratios)
    ratio_errs = np.array(ratio_errs)
    
    allratios.append(ratios)
    allratio_errs.append(ratio_errs)
    
    mean = np.sum((ratios / ratio_errs**2)[:len(masses)]) / np.sum(1/ratio_errs[:len(masses)]**2)
    mean_err = 1/np.sqrt(np.sum(1/ratio_errs[:len(masses)]**2))
        
    plt.errorbar(ftype_names, ratios, ratio_errs, linestyle="none", marker=".", markersize=10)
    plt.axhline(mean, color="red", linestyle="--")
    xlim = plt.xlim()
    plt.fill_between([-0.5, len(variations)+1.5], [mean - mean_err, mean - mean_err], [mean + mean_err, mean + mean_err],
                    color="red", alpha=0.2)
    plt.xlim(xlim)
    plt.title(vnames[iv])
    
    plt.xlabel("Monte Carlo Type")
    plt.ylabel("# Events, Variation / Nominal")
    plt.xticks(rotation=30, ha="right")
    plt.text(0.02, 0.05, "Mean of Scalar Samples", transform=plt.gca().transAxes, color="red")
    plt.tight_layout()
    
    means.append(mean)
    mean_errs.append(mean_err)
    
    if dosave:
        plt.savefig(savedir + "signalbox_detvar_%s.pdf" % v)
        plt.savefig(savedir + "signalbox_detvar_%s.svg" % v)

In [ ]:
for iv, v in enumerate(variations):
    plt.figure(iv)
            
    ratios = []
    ratio_errs = []
    
    this_cvdfs = cvdfs if iv < 7 else cv_escale_dfs
    
    for ift, ft in enumerate(filetypes):
        cv_evt = this_cvdfs[ift].weight.sum()
        cv_evt_var = (this_cvdfs[ift].weight**2).sum()
        
        var_evt = vardfs[ift][iv].weight.sum()
        var_evt_var = (vardfs[ift][iv].weight**2).sum()

        ratio =  var_evt/cv_evt

        ratio_err = np.sqrt(var_evt_var/cv_evt**2 + cv_evt_var*var_evt**2/cv_evt**4)

        ratios.append(ratio)
        ratio_errs.append(ratio_err)
        
    ratios = np.array(ratios)
    ratio_errs = np.array(ratio_errs)
    
    mean = np.sum((ratios / ratio_errs**2)[:len(masses)]) / np.sum(1/ratio_errs[:len(masses)]**2)
    mean_err = 1/np.sqrt(np.sum(1/ratio_errs[:len(masses)]**2))
        
    plt.errorbar(ftype_names, 100*(ratios - 1), 100*ratio_errs, linestyle="none", marker=".", markersize=10)
    plt.axhline(100*(mean - 1), color="red", linestyle="--")
    xlim = plt.xlim()
    plt.fill_between([-0.5, len(variations)+1.5], 
                     100*np.array([mean - mean_err - 1, mean - mean_err - 1]), 
                     100*np.array([mean + mean_err - 1, mean + mean_err - 1]),
                    color="red", alpha=0.2)
    plt.xlim(xlim)
    plt.title(vnames[iv])
    
    plt.xlabel("Monte Carlo Type")
    plt.ylabel("Variation Impact [%]")
    plt.xticks(rotation=30, ha="right")
    plt.text(0.02, 0.05, "Mean of Scalar Samples", transform=plt.gca().transAxes, color="red")
    plt.tight_layout()
    
    if dosave:
        plt.savefig(savedir + "signalbox_detvar_variation_%s.pdf" % v)
        plt.savefig(savedir + "signalbox_detvar_variation_%s.svg" % v)

In [ ]:
# Signal Shape Variation
y = np.sqrt((1 - allratios[0])**2 + (1 - allratios[2])**2/2 + (1 - allratios[3])**2/2)
yerr = np.sqrt(((1 - allratios[0])*allratio_errs[0]/y)**2 +\
    ((1 - allratios[2])*allratio_errs[2]/y/2)**2 +\
    ((1 - allratios[3])*allratio_errs[3]/y/2)**2)

mean = np.sqrt((1 - means[0])**2 + (1 - means[2])**2/2 + (1 - means[3])**2/2)
mean_err = np.sqrt(((1 - means[0])*mean_errs[0]/mean)**2 +\
    ((1 - means[2])*mean_errs[2]/mean/2)**2 +\
    ((1 - means[3])*mean_errs[3]/mean/2)**2)

plt.errorbar(ftype_names, 100*y, 100*yerr, linestyle="none", marker=".", markersize=10)
plt.axhline(100*mean, color="red", linestyle="--")
xlim = plt.xlim()
plt.fill_between([-0.5, len(variations)+1.5], 
                 100*np.array([mean - mean_err, mean - mean_err]), 
                 100*np.array([mean + mean_err, mean + mean_err]),
                color="red", alpha=0.2)
plt.xlim(xlim)
plt.title("Signal Shape Variation")

plt.xlabel("Monte Carlo Type")
plt.ylabel("|Variation Impact| [%]")
plt.xticks(rotation=30, ha="right")
plt.text(0.02, 0.05, "Mean of Scalar Samples", transform=plt.gca().transAxes, color="red")
plt.tight_layout()

if dosave:
    plt.savefig(savedir + "signalbox_detvar_signalshape.pdf")
    plt.savefig(savedir + "signalbox_detvar_signalshape.svg")

In [ ]:
# Signal Shape Variation
y = np.sqrt((1 - allratios[1])**2 + (1 - allratios[5])**2/2 + (1 - allratios[6])**2/2)

yerr = np.sqrt(((1 - allratios[1])*allratio_errs[1]/y)**2 +\
    ((1 - allratios[5])*allratio_errs[5]/y/2)**2 +\
    ((1 - allratios[6])*allratio_errs[6]/y/2)**2)

mean = np.sqrt((1 - means[1])**2 + (1 - means[5])**2/2 + (1 - means[6])**2/2)
mean_err = np.sqrt(((1 - means[1])*mean_errs[1]/mean)**2 +\
    ((1 - means[5])*mean_errs[5]/mean/2)**2 +\
    ((1 - means[6])*mean_errs[6]/mean/2)**2)

plt.errorbar(ftype_names, 100*y, 100*yerr, linestyle="none", marker=".", markersize=10)
plt.axhline(100*mean, color="red", linestyle="--")
xlim = plt.xlim()
plt.fill_between([-0.5, len(variations)+1.5], 
                 100*np.array([mean - mean_err, mean - mean_err]), 
                 100*np.array([mean + mean_err, mean + mean_err]),
                color="red", alpha=0.2)
plt.xlim(xlim)
plt.title("Signal To Noise Variation")

plt.xlabel("Monte Carlo Type")
plt.ylabel("|Variation Impact| [%]")
plt.xticks(rotation=30, ha="right")
plt.text(0.02, 0.05, "Mean of Scalar Samples", transform=plt.gca().transAxes, color="red")
plt.tight_layout()

if dosave:
    plt.savefig(savedir + "signalbox_detvar_s2n.pdf")
    plt.savefig(savedir + "signalbox_detvar_s2n.svg")

In [ ]:
v2mean = {}

for m, v in zip(means, variations):
    print(v, m)
    
    v2mean[v] = m

In [ ]:
# Cal dEdx
err_cal = 100*np.abs(v2mean["nom_calhi"] - v2mean["nom_callo"]) / 2
err_cal

In [ ]:
# Cal Gain
err_gain = 100*np.sqrt(np.mean([(v2mean["nom_gainhi"]-1)**2, (v2mean["nom_gainlo"] -1)**2]))
err_gain

In [ ]:
# Cal MCS
err_mcs = 100*np.sqrt(np.mean([(v2mean["nom_mcslo"]-1)**2, (v2mean["nom_mcshi"] -1)**2]))
err_mcs

In [ ]:
# Total energy scale uncertainty
np.sqrt(err_cal**2 + err_gain**2 + err_mcs**2)

In [ ]:
# Front Ind WC
err_ind0shape = 100*np.sqrt(np.mean([(v2mean["ind0nom"]-1)**2,]))
err_ind0shape

In [ ]:
# Middle Ind Shape
err_ind1shape = 100*np.sqrt(np.mean([(v2mean["ind1bin0"]-1)**2, (v2mean["ind1bin14"] -1)**2]))
err_ind1shape

In [ ]:
err_ind0gain = 100*np.sqrt(np.mean([(v2mean["ind0glo"]-1)**2, (v2mean["ind0ghi"] -1)**2]))
err_ind0gain

In [ ]:
# Total Signal Shape
np.sqrt(err_ind0shape**2 + err_ind1shape**2)

In [ ]:
# Noise
err_noise = 100*np.sqrt(np.mean([(v2mean["noiselhi"]-1)**2,]))
err_noise

In [ ]:
# Total Detector Model
np.sqrt(err_ind0shape**2 + err_ind1shape**2 + err_ind0gain**2 + err_noise**2 )

In [ ]:
# Total Total
np.sqrt(err_ind0shape**2 + err_ind1shape**2 + err_ind0gain**2 + err_noise**2 + err_cal**2 + err_gain**2 + err_mcs**2)

In [ ]:
# Use the loose-cohlike to get the variation on the neutrinos

In [ ]:
v2mean_bkg = {}

for iv, v in enumerate(variations):
    this_cvdfs = cvdfs if iv < 7 else cv_escale_dfs

    cv_evt = this_cvdfs[-1].weight.sum()
    var_evt = vardfs[-1][iv].weight.sum()
    
    v2mean_bkg[v] = var_evt / cv_evt
    print(v, var_evt / cv_evt)

In [ ]:
# Cal dEdx
err_cal_bkg = 100*np.abs(v2mean_bkg["nom_calhi"] - v2mean_bkg["nom_callo"]) / 2
err_cal_bkg

In [ ]:
err_gain_bkg = 100*np.sqrt(np.mean([(v2mean_bkg["nom_gainhi"]-1)**2, (v2mean_bkg["nom_gainlo"] -1)**2]))
err_gain_bkg

In [ ]:
err_mcs_bkg = 100*np.sqrt(np.mean([(v2mean_bkg["nom_mcslo"]-1)**2, (v2mean_bkg["nom_mcshi"] -1)**2]))
err_mcs_bkg

In [ ]:
np.sqrt(err_cal_bkg**2 + err_gain_bkg**2 + err_mcs_bkg**2)

In [ ]:
err_ind0shape_bkg = 100*np.sqrt(np.mean([(v2mean_bkg["ind0nom"]-1)**2,]))
err_ind0shape_bkg

In [ ]:
err_noise_bkg = 100*np.sqrt(np.mean([(v2mean_bkg["noiselhi"]-1)**2,]))
err_noise_bkg

In [ ]:
err_ind1shape_bkg = 100*np.sqrt(np.mean([(v2mean_bkg["ind1bin0"]-1)**2, (v2mean_bkg["ind1bin14"] -1)**2]))
err_ind1shape_bkg

In [ ]:
err_ind0gain_bkg = 100*np.sqrt(np.mean([(v2mean_bkg["ind0glo"]-1)**2, (v2mean_bkg["ind0ghi"] -1)**2]))
err_ind0gain_bkg

In [ ]:
# Total Detector Model
np.sqrt(err_ind0shape_bkg**2 + err_noise_bkg**2 + err_ind1shape_bkg**2 + err_ind0gain_bkg**2)

In [ ]:
# Total Total
np.sqrt(err_ind0shape_bkg**2 + err_noise_bkg**2 + err_ind1shape_bkg**2 + err_ind0gain_bkg**2 + err_cal_bkg**2  + err_gain_bkg**2 + err_mcs_bkg**2)

In [ ]:
# Print the table -- Energy Scale
print(f"""
Calorimetric Gain $\\pm$ 1\\% & {err_gain:.1f} & {err_gain_bkg:.1f} \\\\
Calorimetric $dE/dx$ & {err_cal:.1f} & {err_cal_bkg:.1f} \\\\
Multiple Colomb Scattering & {err_mcs:.1f} & {err_mcs_bkg:.1f} \\\\
\\rowcolor{{gray!10}}
Total Energy Scale & {np.sqrt(err_cal**2 + err_gain**2 + err_mcs**2):.1f} & {np.sqrt(err_cal_bkg**2 + err_gain_bkg**2 + err_mcs_bkg**2):.1f}\\\\
""")

In [ ]:
# Print the table -- Model Variations
print(f"""
Front Induction Signal Shape & {err_ind0shape:.1f} & {err_ind0shape_bkg:.1f}\\\\
Middle Induction Signal Shape & {err_ind1shape:.1f} & {err_ind1shape_bkg:.1f}\\\\
Noise Variations $+10$\\% & {err_noise:.1f} & {err_noise_bkg:.1f}\\\\
Indcution Gain Variation $\\pm 15$\\% & {err_ind0gain:.1f} & {err_ind0gain_bkg:.1f}\\\\
\\rowcolor{{gray!10}}
Total Detector Model & {np.sqrt(err_ind0shape**2 + err_ind1shape**2 + err_ind0gain**2 + err_noise**2 ):.1f} & {np.sqrt(err_ind0shape_bkg**2 + err_noise_bkg**2 + err_ind1shape_bkg**2 + err_ind0gain_bkg**2):.1f}\\\\
""")

In [ ]:
cvdfs[-2].pot.unique().sum()

In [ ]:
cvdfs[-2].shape[0]

In [ ]:
vardfs[-2][0].shape[0]

In [ ]:
vardfs[-2][0].pot.unique().sum()